# Cell segmentation using Cellpose 4.0.6

In [ ]:
#Notebook has been used to establish a working segmentation. For actual segmentation workflow in a batch 
# see notebook "batch_segmentation.ipynb"

#Pipeline overview (Fura-2 Ca2+ imaging): Before this notebook

#1. Open raw .tif time stack in Fiji (interleaved 340/380 frames).
#2. Deinterleave stack into 340 nm and 380 nm stacks.
#4. Compute ratio stack: R = F340 / F380, 32-bit
#5. Select baseline time window (frames 100-200) and average frames → segmentation image.
#6. Save in folder for segmentation with same name, put avgforseg_Slicex_ in beginning

#6. Segment cells on segmentation image → generate ROIs.
#7. Apply ROIs to ratio stack → extract mean ROI traces over time.
#8. Normalize traces to baseline (ΔR/R0).
#9. Plot Ca2+ responses across AngII conditions.




In [3]:
import numpy as np
from skimage.io import imread
import imageio.v2 as iio
import matplotlib.pyplot as plt
import skimage
from skimage.segmentation import mark_boundaries
from skimage.exposure import rescale_intensity
import matplotlib.patches as patches
from skimage.measure import label, regionprops_table
from cellpose import io, models
from cellpose.io import imread
import os

print("imports ok")

#prepping everything and load the required packages

imports ok


In [ ]:
# Define the main path
#get a list of possible inputs

path = r"D:/FA_Data_in_processing"

# Collect and sort files
files = sorted(os.listdir(path), key=str.lower)

#Print  them with indices
for i, f in enumerate(files):
   print(f"{i}: {f}")

0: avgforseg_16_Slice1_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice1_3_MMStack_Default.ome.tif #1-1.tif
1: avgforseg_16_Slice2_251110_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_100pM_500pM_Slice2_1h_1_MMStack_Default.ome.tif #2-1.tif
2: avgforseg_16iinv_Slice3_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_80pM_200pM_500pM_Slice3_1_MMStack_Default.ome.tif #1-1.tif
3: avgforseg_16inv_Slice1_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice1_2_MMStack_Default.ome.tif #1-1.tif
4: avgforseg_16inv_Slice1_251113_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20.tif
5: avgforseg_16inv_Slice2251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice2_1_MMStack_Default.ome.tif #1-1.tif
6: avgforseg_16inv_Slice2_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice2_1_MMStack_Default.ome.tif #1-1.tif
7: Avgforseg_16inv_Slice2_251113_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM.tif
8: avgforseg_16inv_Slice3_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Bas

In [14]:
#Setup Cellpose and read images

io.logger_setup()

# Define the model
model = models.CellposeModel(gpu=False) #gpu doesnt help with reading now

# Read image files

index = 4 #choose an image, order should match explorer if ordered A-Z
if index >= len(files):
    raise IndexError(f"Only {len(files)} files available")

img_file = os.path.join(path, files[index])


#img_file = os.path.join(path, "avgforseg_slice3_251113_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_100pM_500pM_Slice3_1h_1_MMStack_Default.ome.tif #1-1.tif")

#check image scales, expected (270,4#80) float32
print(img0.shape, img0.dtype)

#check filesize
print(os.path.getsize(img_file)/1e9, "GB")

img0 = iio.imread(img_file)


creating new log file
2026-01-05 20:04:40,972 [INFO] WRITING LOG OUTPUT TO C:\Users\caraj\.cellpose\run.log
2026-01-05 20:04:40,973 [INFO] 
cellpose version: 	4.0.8 
platform:       	win32 
python version: 	3.10.8 
torch version:  	2.9.1+cpu
2026-01-05 20:04:40,974 [INFO] >>>> using CPU
2026-01-05 20:04:40,975 [INFO] >>>> using CPU
2026-01-05 20:04:42,550 [INFO] >>>> loading model C:\Users\caraj\.cellpose\models\cpsam
(270, 480) uint16
0.000259384 GB


In [ ]:
#prepare script for all files in segmentation folder

path = r"D:\FA_Data_in_processing"
#print(os.listdir(path))
img_file = os.path.join(path, 'test2.jpg' )
#print(img_file)
img0 = iio.imread(img_file)
model = models.CellposeModel(gpu=False)
#masks0, flows0, styles0, diams0 = model.eval(
#    img0,
#    channels=None, #grayscale
#    diameter=30 #not zero!
#)

#original code:
masks0, flows0, styles0 = model.eval(img0)

In [ ]:
#ROIs speichern für Inport in Fiji

from cellpose import utils
io.save_rois(masks0, '250604_Coverslip2_Rec2')


NameError: name 'masks0' is not defined

In [16]:
# Normalize ratio image for stable Cellpose behavior
# - use robust percentiles (1–99%) instead of min/max
# - rescale intensities to [0, 1] without changing cell shapes

img0c = img0.astype(np.float32, copy=True)
p1, p99 = np.percentile(img0c, (1, 99))
img0n = (img0c - p1) / (p99 - p1)
img0n = np.clip(img0n, 0, 1)


In [ ]:

# Segmentation   simple

model = models.CellposeModel(gpu=True)
#masks0, flows0, styles0, diams0 = model.eval(
#    img0,
#    channels=None, #grayscale
#    diameter=30 #not zero!
#)

#original code:
masks0, flows0, styles0 = model.eval(img0)

2026-01-05 20:09:28,236 [INFO] Neither TORCH CUDA nor MPS version not installed/working.
2026-01-05 20:09:28,238 [INFO] >>>> using CPU
2026-01-05 20:09:28,239 [INFO] >>>> using CPU
2026-01-05 20:09:29,979 [INFO] >>>> loading model C:\Users\caraj\.cellpose\models\cpsam
2026-01-05 20:09:30,497 [INFO] processing grayscale image with (270, 480) HW


KeyboardInterrupt: 

In [ ]:
print(type(img0), img0.shape, img0.dtype, img0.ndim)

#make cpsam lighter/faster ? try
masks, flows, styles, diams = model.eval(
    img0n,
    channels=None,
    diameter=25,         # fixed, don't use None/0
    augment=False,
    niter=200,           # reduce iterations
    batch_size=1,
)

<class 'numpy.ndarray'> (270, 480) uint16 2
2026-01-05 20:11:56,567 [INFO] processing grayscale image with (270, 480) HW


KeyboardInterrupt: 

In [ ]:

#New cell
# Check Cellpose API and validate segmentation image
# - confirm which Cellpose class exists in this version
# - ensure image is 2D, numeric, and free of NaNs/Infs
# - avoid debugging segmentation when the input image is broken

from cellpose import models
print ([x for x in dir(models) if 'Cellpose' in x])

# checking on all important image parameter
#float divided by zero, searching for the error 
print("shape:", img0.shape, "dtype:", img0.dtype)
print("vmin/max:", np.nanmin(img0), np.nanmax(img0))
print("nan count:", np.isnan(img0).sum(), "inf count:", np.isinf(img0).sum())
print("std:", np.nanstd(img0))    

['CellposeModel']
shape: (270, 480) dtype: uint16
vmin/max: 0 65535
nan count: 0 inf count: 0
std: 8930.422719399809


In [5]:
#Rename my average for segmentation images
path = r"D:/FA_Data_in_processing/Average_for_Segmentation"

# Collect and sort files
files = sorted(
    [f for f in os.listdir(path)
     if f.lower().endswith(".tif") and os.path.isfile(os.path.join(path, f))],
    key=str.lower
)

#Print  them with indices
for i, f in enumerate(files):
   print(f"{i}: {f}")

0: avgforseg_16_Slice1_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice1_3_MMStack_Default.ome.tif #1-1.tif
1: avgforseg_16_Slice2_251110_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_100pM_500pM_Slice2_1h_1_MMStack_Default.ome.tif #2-1.tif
2: avgforseg_16iinv_Slice3_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_80pM_200pM_500pM_Slice3_1_MMStack_Default.ome.tif #1-1.tif
3: avgforseg_16inv_Slice1_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice1_2_MMStack_Default.ome.tif #1-1.tif
4: avgforseg_16inv_Slice1_251113_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20.tif
5: avgforseg_16inv_Slice2251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice2_1_MMStack_Default.ome.tif #1-1.tif
6: avgforseg_16inv_Slice2_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice2_1_MMStack_Default.ome.tif #1-1.tif
7: Avgforseg_16inv_Slice2_251113_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM.tif
8: avgforseg_16inv_Slice3_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Bas

In [7]:
for i, fname in enumerate(files):
    old_path = os.path.join(path, fname)
    new_name = f"{i:02d}_{fname}"   # 00_, 01_, 02_, ...
    new_path = os.path.join(path, new_name)

    os.rename(old_path, new_path)

    for i, fname in enumerate(files):
        print(f"{fname}  ->  {i:02d}_{fname}")

avgforseg_16_Slice1_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice1_3_MMStack_Default.ome.tif #1-1.tif  ->  00_avgforseg_16_Slice1_251029_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_200pM_500pM_Slice1_3_MMStack_Default.ome.tif #1-1.tif
avgforseg_16_Slice2_251110_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_100pM_500pM_Slice2_1h_1_MMStack_Default.ome.tif #2-1.tif  ->  01_avgforseg_16_Slice2_251110_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_20pM_100pM_500pM_Slice2_1h_1_MMStack_Default.ome.tif #2-1.tif
avgforseg_16iinv_Slice3_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_80pM_200pM_500pM_Slice3_1_MMStack_Default.ome.tif #1-1.tif  ->  02_avgforseg_16iinv_Slice3_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_80pM_200pM_500pM_Slice3_1_MMStack_Default.ome.tif #1-1.tif
avgforseg_16inv_Slice1_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500pM_Slice1_2_MMStack_Default.ome.tif #1-1.tif  ->  03_avgforseg_16inv_Slice1_251030_Hac15_Co2_AngReihe_4.5mMKBBS_Baseline_50pM_200pM_500p

In [8]:
#also rename in cellpose_roi subfolder
import os

base_path = r"D:/FA_Data_in_processing/Average_for_Segmentation"
rois_path = os.path.join(base_path, "cellpose_rois")

# collect numbered tif files in main folder
main_files = sorted(
    [f for f in os.listdir(base_path)
     if f.lower().endswith(".tif") and os.path.isfile(os.path.join(base_path, f))],
    key=str.lower
)

# build mapping: original_name -> index
index_map = {}

for f in main_files:
    idx, rest = f.split("_", 1)   # "00", "avgforseg_..."
    index_map[rest] = idx


In [15]:
#renaming of subfolder entries, adding same indices as above 00,01,02 etc. 
#will make subsequent work easier i guess.

import os
import re

base_path = r"D:/FA_Data_in_processing/Average_for_Segmentation"
rois_path = os.path.join(base_path, "cellpose_rois")

def strip_leading_index(name):
    if "_" in name and name.split("_", 1)[0].isdigit():
        return name.split("_", 1)[1]
    return name

def normalize_for_matching(name):
    name = strip_leading_index(os.path.basename(name))
    name_low = name.lower()
    # remove cellpose suffixes right before extension
    name_low = re.sub(r'(_masks|_rois_rois|_rois)(?=\.)', '', name_low)
    return name_low

# build index map from main folder numbered tifs
main_files = sorted(
    [f for f in os.listdir(base_path)
     if f.lower().endswith(".tif") and os.path.isfile(os.path.join(base_path, f))],
    key=str.lower
)

index_map = {}
for f in main_files:
    idx, rest = f.split("_", 1)   # "00", "avgforseg_..."
    index_map[rest.lower()] = idx

def rename_cellpose_rois(do_rename=True):
    for root, _, files in os.walk(rois_path):
        for f in files:
            old_path = os.path.join(root, f)

            f_clean = strip_leading_index(f)
            f_norm = normalize_for_matching(f_clean)

            base_no_ext, _ = os.path.splitext(f_norm)
            candidate_key = base_no_ext + ".tif"   # match against original tif name

            idx = index_map.get(candidate_key)
            if idx is None:
                print(f"⚠️ no match for: {old_path}")
                continue

            new_name = f"{idx}_{f_clean}"
            new_path = os.path.join(root, new_name)

            # already correct?
            if old_path == new_path:
                continue

            # collision check
            if os.path.exists(new_path):
                print(f"❌ target exists, skipping: {new_path}")
                continue

            print(f"{old_path}  ->  {new_path}")
            if do_rename:
                os.rename(old_path, new_path)

# RUN IT:
rename_cellpose_rois(do_rename=True)
